In [ ]:
import pandas as pd

museum_df = pd.read_csv("museum.csv")
print(museum_df.head())

queen_park_df = pd.read_csv("queen_park.csv")
print(queen_park_df.head())

st_patrick_df = pd.read_csv("st_patrick.csv")
print(st_patrick_df.head())

spadina_df = pd.read_csv("spadina.csv")
print(spadina_df.head())

king_df = pd.read_csv("king.csv")
print(king_df.head())

   Unnamed: 0                  Start-End  \
0           0     ('Museum', 'Bathurst')   
1           1          ('Museum', 'Bay')   
2           2  ('Museum', 'Dundas West')   
3           3  ('Museum', 'St. Patrick')   
4           4  ('Museum', 'Main Street')   

                                       Shortest Path  Total Distance  \
0                                                NaN             NaN   
1            ['Museum', 'start_72', 'end_72', 'Bay']      466.894346   
2  ['Museum', 'start_76', 'end_76', 'St. George',...     6205.065911   
3  ['Museum', 'start_180', 'end_180', "Queen's Pa...     1426.881684   
4  ['Museum', 'start_4206', 'end_4206', 'Bay', 's...     8106.972523   

        Time Time Unit                                  Combo_G  
0        NaN       NaN                                      NaN  
1   8.703463       min     DiGraph with 253 nodes and 288 edges  
2   1.927829         h   DiGraph with 3193 nodes and 4503 edges  
3  26.598761       min    DiGraph with

after getting files, count number of "no paths found" or None or something against everything and get the percentage

In [ ]:
# Combine all the dataframes into a single dataframe
all_dfs = pd.concat([museum_df, queen_park_df, st_patrick_df, spadina_df, king_df])

# Count the number of rows where the 'Shortest Path' column is NaN (No path found)
no_path_count = all_dfs['Shortest Path'].isna().sum()

# Calculate the total number of paths (rows)
total_paths = len(all_dfs)

# Calculate the percentage of "No Paths Found"
no_path_percentage = (no_path_count / total_paths) * 100

# Display the results
results = {
    "Total Paths": total_paths,
    "No Paths Found": no_path_count,
    "Percentage of No Paths": f"{no_path_percentage:.2f}%"
}

print(results)


{'Total Paths': 271, 'No Paths Found': 30, 'Percentage of No Paths': '11.07%'}


In [ ]:
print("Percentage of paths found: ", 100-no_path_percentage, "%")

Percentage of paths found:  88.92988929889299 %


after getting files, get the time column, see if can compare against something

In [ ]:
museum_plus_google_maps_df = pd.read_csv("museum_plus_google_maps.csv")
print(museum_plus_google_maps_df.head())

   Unnamed: 0                  Start-End  \
0           0     ('Museum', 'Bathurst')   
1           1          ('Museum', 'Bay')   
2           2  ('Museum', 'Dundas West')   
3           3  ('Museum', 'St. Patrick')   
4           4  ('Museum', 'Main Street')   

                                       Shortest Path  Total Distance  \
0                                                NaN             NaN   
1            ['Museum', 'start_72', 'end_72', 'Bay']      466.894346   
2  ['Museum', 'start_76', 'end_76', 'St. George',...     6205.065911   
3  ['Museum', 'start_180', 'end_180', "Queen's Pa...     1426.881684   
4  ['Museum', 'start_4206', 'end_4206', 'Bay', 's...     8106.972523   

        Time Time Unit                                  Combo_G  \
0        NaN       NaN                                      NaN   
1   8.703463       min     DiGraph with 253 nodes and 288 edges   
2   1.927829         h   DiGraph with 3193 nodes and 4503 edges   
3  26.598761       min    DiGraph 

In [ ]:
# Add a new column for walking time (hours)
museum_plus_google_maps_df['Walking Time (hours)'] = museum_plus_google_maps_df['Total Distance'] / 1000 / 5  # Convert meters to kilometers and divide by speed (5 km/h)

# If you want the time in minutes instead of hours
museum_plus_google_maps_df['Walking Time (minutes)'] = museum_plus_google_maps_df['Walking Time (hours)'] * 60

# Display the updated DataFrame
print(museum_plus_google_maps_df)


    Unnamed: 0                                  Start-End  \
0            0                     ('Museum', 'Bathurst')   
1            1                          ('Museum', 'Bay')   
2            2                  ('Museum', 'Dundas West')   
3            3                  ('Museum', 'St. Patrick')   
4            4                  ('Museum', 'Main Street')   
5            5                         ('Museum', 'Pape')   
6            6                      ('Museum', 'Coxwell')   
7            7                     ('Museum', 'Woodbine')   
8            8                   ('Museum', 'St. George')   
9            9                       ('Museum', 'Wilson')   
10          10                    ('Museum', 'St. Clair')   
11          11                     ('Museum', 'Eglinton')   
12          12                     ('Museum', 'Lawrence')   
13          13                   ('Museum', 'York Mills')   
14          14                        ('Museum', 'Finch')   
15          15          

In [ ]:
# Convert Time to minutes based on the time unit (Walking Time)
museum_plus_google_maps_df['WalkingTime_in_min'] = museum_plus_google_maps_df.apply(
    lambda row: row['Walking Time (hours)'] * 60 if pd.notnull(row['Walking Time (hours)']) and row['Time Unit'] == 'h' else row['Walking Time (minutes)'],
    axis=1
)

# Convert GoogleMapsTime to minutes based on the time unit
museum_plus_google_maps_df['GoogleMapsTime_in_min'] = museum_plus_google_maps_df.apply(
    lambda row: row['GoogleMapsTime'] * 60 if row['Google Time Unit'] == 'h' else row['GoogleMapsTime'],
    axis=1
)

# Calculate the absolute difference
museum_plus_google_maps_df['Difference_in_min'] = abs(museum_plus_google_maps_df['WalkingTime_in_min'] - museum_plus_google_maps_df['GoogleMapsTime_in_min'])

# Check if the difference is within ±3 minutes
museum_plus_google_maps_df['Within_3_min'] = museum_plus_google_maps_df['Difference_in_min'] <= 3

# Calculate the proportion
proportion_within_3_min = museum_plus_google_maps_df['Within_3_min'].mean()

print(f"Proportion within ±3 minutes: {proportion_within_3_min:.2%}")


Proportion within ±3 minutes: 12.96%


In [ ]:
# Convert Walking Time to minutes (similar to previous approach)
museum_plus_google_maps_df['WalkingTime_in_min'] = museum_plus_google_maps_df.apply(
    lambda row: row['Walking Time (hours)'] * 60 if pd.notnull(row['Walking Time (hours)']) and row['Time Unit'] == 'h' else row['Walking Time (minutes)'],
    axis=1
)

# Convert GoogleMapsTime to minutes
museum_plus_google_maps_df['GoogleMapsTime_in_min'] = museum_plus_google_maps_df.apply(
    lambda row: row['GoogleMapsTime'] * 60 if row['Google Time Unit'] == 'h' else row['GoogleMapsTime'],
    axis=1
)

# Calculate the absolute difference in minutes
museum_plus_google_maps_df['Difference_in_min'] = abs(museum_plus_google_maps_df['WalkingTime_in_min'] - museum_plus_google_maps_df['GoogleMapsTime_in_min'])

# Filter rows within ±3 minutes
museum_plus_google_maps_df['Within_3_min'] = museum_plus_google_maps_df['Difference_in_min'] <= 3

# Filter rows where distances are less than or equal to GoogleMapsDistance (converted to meters)
museum_plus_google_maps_df['Distance_LE_Google'] = museum_plus_google_maps_df['Total Distance'] <= (museum_plus_google_maps_df['GoogleMapsDistance (km)'] * 1000)

# Combine the two conditions: within ±3 minutes and distance less than or equal to Google Maps distance
museum_plus_google_maps_df['Satisfies_Both'] = museum_plus_google_maps_df['Within_3_min'] & museum_plus_google_maps_df['Distance_LE_Google']

# Calculate the proportion of rows that satisfy both conditions
proportion_satisfies_both = museum_plus_google_maps_df['Satisfies_Both'].sum() / museum_plus_google_maps_df['Within_3_min'].sum()

print(f"Proportion of rows within ±3 minutes that also have distances ≤ GoogleMapsDistance: {proportion_satisfies_both:.2%}")


Proportion of rows within ±3 minutes that also have distances ≤ GoogleMapsDistance: 28.57%


In [ ]:
# Calculate the proportion directly
proportion_distance_le_google = (
    (museum_plus_google_maps_df['Total Distance'] <= (museum_plus_google_maps_df['GoogleMapsDistance (km)'] * 1000)).sum()
    / len(museum_plus_google_maps_df)
)

print(f"Proportion of rows where Total Distance ≤ GoogleMapsDistance: {proportion_distance_le_google:.2%}")


Proportion of rows where Total Distance ≤ GoogleMapsDistance: 48.15%


In [ ]:
# Filter rows where the time difference is within ±3 minutes
time_diff_filter = (
    (museum_plus_google_maps_df['Time Unit'] == 'min') &
    (abs(museum_plus_google_maps_df['Time'] - museum_plus_google_maps_df['GoogleMapsTime']) <= 3)
) | (
    (museum_plus_google_maps_df['Time Unit'] == 'h') &
    (abs(museum_plus_google_maps_df['Time'] * 60 - museum_plus_google_maps_df['GoogleMapsTime']) <= 3)
)

filtered_df = museum_plus_google_maps_df[time_diff_filter]

# Check the condition: Total Distance < 1000 meters
distance_filter = filtered_df['Total Distance'] < 1000

# Calculate the proportion
proportion_under_1000m = distance_filter.mean()

# Output the result
print(f"Proportion of rows within ±3 minutes of Google Maps time with distances < 1000 meters: {proportion_under_1000m:.2%}")


Proportion of rows within ±3 minutes of Google Maps time with distances < 1000 meters: 50.00%


In [ ]:
# Convert Time to minutes where necessary
museum_plus_google_maps_df['Time_in_min'] = museum_plus_google_maps_df.apply(
    lambda row: row['Time'] * 60 if row['Time Unit'] == 'h' else row['Time'], axis=1
)

# Convert GoogleMapsTime to minutes where necessary
museum_plus_google_maps_df['GoogleMapsTime_in_min'] = museum_plus_google_maps_df.apply(
    lambda row: row['GoogleMapsTime'] * 60 if row['Google Time Unit'] == 'h' else row['GoogleMapsTime'], axis=1
)

# Filter rows where time is within ±3 minutes of Google Maps Time
filtered_df = museum_plus_google_maps_df[
    (museum_plus_google_maps_df['Time_in_min'].between(
        museum_plus_google_maps_df['GoogleMapsTime_in_min'] - 3,
        museum_plus_google_maps_df['GoogleMapsTime_in_min'] + 3
    ))
]

# Display the filtered DataFrame
filtered_df


,Unnamed: 0,Start-End,Shortest Path,Total Distance,Time,Time Unit,Combo_G,GoogleMapsTime,Google Time Unit,GoogleMapsDistance (km),Walking Time (hours),Walking Time (minutes),WalkingTime_in_min,GoogleMapsTime_in_min,Difference_in_min,Within_3_min,Distance_LE_Google,Satisfies_Both,Time_in_min
8,8,"('Museum', 'St. George')","['Museum', 'start_17', 'end_17', 'St. George']",500.310826,9.326385,min,DiGraph with 137 nodes and 120 edges,7.0,min,0.5,0.100062,6.003730,6.003730,7.0,0.996270,True,False,False,9.326385
20,20,"('Museum', 'Summerhill')","['Museum', 'start_150', 'end_150', 'Bay', 'sta...",1777.532934,33.135315,min,DiGraph with 449 nodes and 590 edges,31.0,min,2.2,0.355507,21.330395,21.330395,31.0,9.669605,False,True,False,33.135315


In [ ]:
display_df = filtered_df.drop(columns=['Unnamed: 0', 'Difference_in_min', 'Within_3_min', 'Walking Time (hours)', 'Distance_LE_Google', 'Satisfies_Both'], inplace = False)
display_df

,Start-End,Shortest Path,Total Distance,Time,Time Unit,Combo_G,GoogleMapsTime,Google Time Unit,GoogleMapsDistance (km),Walking Time (minutes),WalkingTime_in_min,GoogleMapsTime_in_min,Time_in_min
8,"('Museum', 'St. George')","['Museum', 'start_17', 'end_17', 'St. George']",500.310826,9.326385,min,DiGraph with 137 nodes and 120 edges,7.0,min,0.5,6.003730,6.003730,7.0,9.326385
20,"('Museum', 'Summerhill')","['Museum', 'start_150', 'end_150', 'Bay', 'sta...",1777.532934,33.135315,min,DiGraph with 449 nodes and 590 edges,31.0,min,2.2,21.330395,21.330395,31.0,33.135315


## new times for walking (based on 5km/h walking time)

In [ ]:
# Add a new column for walking time (hours)
museum_plus_google_maps_df['Walking Time (hours)'] = museum_plus_google_maps_df['Total Distance'] / 1000 / 5  # Convert meters to kilometers and divide by speed (5 km/h)

# If you want the time in minutes instead of hours
museum_plus_google_maps_df['Walking Time (minutes)'] = museum_plus_google_maps_df['Walking Time (hours)'] * 60

# Display the updated DataFrame
print(museum_plus_google_maps_df)


    Unnamed: 0                                  Start-End  \
0            0                     ('Museum', 'Bathurst')   
1            1                          ('Museum', 'Bay')   
2            2                  ('Museum', 'Dundas West')   
3            3                  ('Museum', 'St. Patrick')   
4            4                  ('Museum', 'Main Street')   
5            5                         ('Museum', 'Pape')   
6            6                      ('Museum', 'Coxwell')   
7            7                     ('Museum', 'Woodbine')   
8            8                   ('Museum', 'St. George')   
9            9                       ('Museum', 'Wilson')   
10          10                    ('Museum', 'St. Clair')   
11          11                     ('Museum', 'Eglinton')   
12          12                     ('Museum', 'Lawrence')   
13          13                   ('Museum', 'York Mills')   
14          14                        ('Museum', 'Finch')   
15          15          